# 05 - Regression

###### Here, to build a predictive model able to anticipate possible changes and additions in the dataset (in this case, the finished operations historical) a regression model was developed. The regression model seeks to identify patterns in the current dataset and predict the behaviour of additional informations (related to finished operations).

##### Considered Regression Models:

###### - Linear Regression;
###### - Ridge;
###### - Lasso;
###### - ElasticNet;
###### - Decision Tree;
###### - Random Forest;
###### - Gradient Boosting;
###### - XGBoost;
###### - Support Vector Regressor (SVR)
    
----

#### Loading Packages and Database
----

In [1]:
import plotly.io as pio
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
# import fast_dummies
from sklearn.preprocessing import OneHotEncoder
from scipy import stats
import os
import sys
from pathlib import Path

In [2]:
# Adicionar o diretório 'modules_py' ao sys.path
modules_path = Path("modules_py").resolve()
sys.path.append(str(modules_path))

from data_cleaning import DataCleaning_MyData, DataCleaning_GroupData, DataCleaning_OffCod, DataCleaning_OffCod_NotNA, DataCleaning_frequents, DataCleaning_dummies
from aplicaRegressao import aplicar_modelos_regressao

try:
    # Executar a função principal de limpeza de dados
    my_data = DataCleaning_MyData()
    clean_media = DataCleaning_GroupData()
    OffCod_Agru = DataCleaning_OffCod()
    OffCod_Agru_NotNA = DataCleaning_OffCod_NotNA()
    mais_frequentes = DataCleaning_frequents()
    dummies = DataCleaning_dummies()
    
    print("Dados processados com sucesso!")
    display(OffCod_Agru_NotNA)
    

# Mostrar o erro caso não execute
except Exception as e:
    print(f"Ocorreu um erro: {e}")

Dados processados com sucesso!


,OFFCod,Andrea_DEPARTAMENTO.Descricao,TipodeProduto,Produto.Nome,Poder Linear,Poder Z,SomaDeHorasApontadasUnitario
5,71,CONFORMACAO,Caldeira,CQOG,66.000000,156.000000,13513.50
6,71,MONTAGEM,Caldeira,CQOG,66.000000,156.000000,12136.00
7,71,SOLDA,Caldeira,CQOG,66.000000,156.000000,6920.00
8,71,TRACAGEM,Caldeira,CQOG,66.000000,156.000000,22777.50
9,71,USINAGEM,Caldeira,CQOG,66.000000,156.000000,13788.00
...,...,...,...,...,...,...,...
312,160039,TRACAGEM,Caldeira,CRCHRSG,27.941176,-43.820807,3888.00
313,160039,USINAGEM,Caldeira,CRCHRSG,27.941176,-43.820807,8380.00
384,810390,MONTAGEM,Precipitador,PRECIPITADOR ELETROSTATICO,41.071429,53.379421,23.42
385,810390,SOLDA,Precipitador,PRECIPITADOR ELETROSTATICO,41.071429,53.379421,423.74


In [3]:
# Usando o filtro do OffCod
OffCod = OffCod_Agru[OffCod_Agru["SomaDeHorasApontadasUnitario"].notna()]
OffCod = OffCod[OffCod["SomaDeHorasApontadasUnitario"] != 0]

# Removendo colunas com dados faltantes
OffCod = OffCod.drop(['Poder Linear', 'Poder Z'], axis= 1)

# Lista de departamentos
departamentos_list = ['CONFORMACAO', 'MONTAGEM', 'SOLDA', 'TRACAGEM', 'USINAGEM']

# Dicionário para armazenar os DataFrames
Departamento = {}

# Criando os DataFrames com um loop
for d in departamentos_list:
    Departamento[d] = OffCod[
        (OffCod['TipodeProduto'] == 'Caldeira') &
        (OffCod['Andrea_DEPARTAMENTO.Descricao'] == d)
    ]

    # Exibir o número de registros para cada departamento
    print(f"Departamento: {d}, Registros: {len(Departamento[d])}")

# Opção 1: Criando uma lista de dataframes para cada departamento
# dfs = [
#     Departamento['CONFORMACAO'],
#     Departamento['MONTAGEM'],
#     Departamento['SOLDA'],
#     Departamento['TRACAGEM'],
#     Departamento['USINAGEM']
#     ]

# Opção 2: Utilizando um único dataframe
dfs = OffCod_Agru_NotNA

display(Departamento['CONFORMACAO'])

# teste
# display(Departamento['CONFORMACAO'])

# display(dfs)

OffCod_Agru_NotNA.dtypes

Departamento: CONFORMACAO, Registros: 46
Departamento: MONTAGEM, Registros: 56
Departamento: SOLDA, Registros: 55
Departamento: TRACAGEM, Registros: 53
Departamento: USINAGEM, Registros: 53


,OFFCod,Andrea_DEPARTAMENTO.Descricao,TipodeProduto,Produto.Nome,SomaDeHorasApontadasUnitario
0,66,CONFORMACAO,Caldeira,CRCHRSG,7066.00
5,71,CONFORMACAO,Caldeira,CQOG,13513.50
26,115,CONFORMACAO,Caldeira,CRCHRSG,1823.00
31,117,CONFORMACAO,Caldeira,CRQ,604.00
44,132,CONFORMACAO,Caldeira,CRQ,1353.00
49,133,CONFORMACAO,Caldeira,CQOG,6899.00
54,134,CONFORMACAO,Caldeira,CC,191.00
59,135,CONFORMACAO,Caldeira,CRCHRSG,7066.00
72,138,CONFORMACAO,Caldeira,CC,328.00
77,139,CONFORMACAO,Caldeira,CND,2715.00


OFFCod                           category
Andrea_DEPARTAMENTO.Descricao    category
TipodeProduto                    category
Produto.Nome                     category
Poder Linear                      float64
Poder Z                           float64
SomaDeHorasApontadasUnitario      float64
dtype: object

In [4]:
print(type(dfs))

<class 'pandas.core.frame.DataFrame'>


In [5]:
resultados_onehot = aplicar_modelos_regressao(dfs, target_column='SomaDeHorasApontadasUnitario', encoder='onehot')

resultados_label = aplicar_modelos_regressao(dfs, target_column='SomaDeHorasApontadasUnitario', encoder='label')

resultados_combinados = pd.concat([resultados_onehot, resultados_label], ignore_index=True)

# Exportar resultados para Excel
resultados_combinados.to_excel('resultados_regressao.xlsx', index=False)

# Exibir os resultados
print(resultados_combinados)

Processando dataset 1 com 'onehot'


C:\Users\pedro\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:658: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 168550912.22941828, tolerance: 3791694.4775571064
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\pedro\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:658: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2891307992.562075, tolerance: 4017629.4095871584
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\pedro\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:658: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1007504368.9559221, tolerance: 4037957.7804305805
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\pedro\AppData\Roamin

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000118 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 44
[LightGBM] [Info] Number of data points in the train set: 78, number of used features: 4
[LightGBM] [Info] Start training from score 12584.797024
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -